In [1]:
import random
import time
import torch
import numpy as np
import ray
from worker import Learner, Actor, ReplayBuffer
import config

/home/timorius/Studium/Semester2/PR-PAMI/doomenv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/timorius/Studium/Semester2/PR-PAMI/doomenv/lib/python3.8/site-packages/gym/envs/registration.py:517: UserWarning: WARN: Overriding environment VizdoomBasic-v0
  logger.warn(f"Overriding environment {spec.id}")
/home/timorius/Studium/Semester2/PR-PAMI/doomenv/lib/python3.8/site-packages/gym/envs/registration.py:517: UserWarning: WARN: Overriding environment VizdoomCorridor-v0
  logger.warn(f"Overriding environment {spec.id}")
/home/timorius/Studium/Semester2/PR-PAMI/doomenv/lib/python3.8/site-packages/gym/envs/registration.py:517: UserWarning: WARN: Overriding environment VizdoomDefendCenter-v0
  logger.warn(f"Overriding environment {spec.id}")
/home/timorius/Studium/Semester2/PR-PAMI/doomenv/lib/python3

In [2]:
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)
torch.set_num_threads(1)

In [3]:
def get_epsilon(actor_id: int, base_eps: float = config.base_eps, alpha: float = config.alpha, num_actors: int = config.num_actors):
    exponent = 1 + actor_id / (num_actors-1) * alpha
    return base_eps**exponent

In [4]:
def train(num_actors=config.num_actors, log_interval=config.log_interval):
    ray.init()

    buffer = ReplayBuffer.remote()
    learner = Learner.remote(buffer)
    actors = [Actor.remote(get_epsilon(i), learner, buffer) for i in range(num_actors)]

    for actor in actors:
        actor.run.remote()

    while not ray.get(buffer.ready.remote()):
        time.sleep(log_interval)
        ray.get(buffer.log.remote(log_interval))
        print()

    print('start training')
    learner.run.remote()
    
    done = False
    while not done:
        time.sleep(log_interval)
        done = ray.get(buffer.log.remote(log_interval))
        print()

In [ ]:
train()

(scheduler +15s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(scheduler +15s) Error: No available node types can fulfill resource request {'GPU': 1.0, 'CPU': 1.0}. Add suitable node types to this cluster to resolve this issue.


(pid=8150) /home/timorius/Studium/Semester2/PR-PAMI/doomenv/lib/python3.8/site-packages/gym/envs/registration.py:517: UserWarning: WARN: Overriding environment VizdoomBasic-v0
(pid=8150)   logger.warn(f"Overriding environment {spec.id}")
(pid=8150) /home/timorius/Studium/Semester2/PR-PAMI/doomenv/lib/python3.8/site-packages/gym/envs/registration.py:517: UserWarning: WARN: Overriding environment VizdoomCorridor-v0
(pid=8150)   logger.warn(f"Overriding environment {spec.id}")
(pid=8150) /home/timorius/Studium/Semester2/PR-PAMI/doomenv/lib/python3.8/site-packages/gym/envs/registration.py:517: UserWarning: WARN: Overriding environment VizdoomDefendCenter-v0
(pid=8150)   logger.warn(f"Overriding environment {spec.id}")
(pid=8150) /home/timorius/Studium/Semester2/PR-PAMI/doomenv/lib/python3.8/site-packages/gym/envs/registration.py:517: UserWarning: WARN: Overriding environment VizdoomDefendLine-v0
(pid=8150)   logger.warn(f"Overriding environment {spec.id}")
(pid=8150) /home/timorius/Studium